In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import linalg

# Repaso - Matrices Python

In [2]:
prueba = np.asmatrix(np.array([[8, 14, -6], [12,7,4], [-11,3,21]]))

In [3]:
pd.DataFrame(prueba)

,0,1,2
0,8,14,-6
1,12,7,4
2,-11,3,21


In [4]:
prueba[:,1] #Columna
prueba[1,:] #Fila

matrix([[12,  7,  4]])

In [5]:
eig_vals, eig_vecs = np.linalg.eig(prueba)

In [6]:
eig_vals[1]*eig_vecs[:,1], prueba@eig_vecs[:,1]

(matrix([[-8.7841731 ],
         [-0.64592692],
         [23.43943856]]),
 matrix([[-8.7841731 ],
         [-0.64592692],
         [23.43943856]]))

# Funciones SSH:

In [7]:
def H_t1_t2(N, t1, t2, PBC = False):
    Matriz = np.zeros((N,N))

    a = 0 #diagonal
    b = -t2 #1 + delta #la que aparece despues - en este caso sería -t2
    c = t1 #1 - delta #la que aparece primero - en este caso sería t1

    for i in range(N):
        Matriz[i,i] = a
    for i in range(N-1):
        if (i%2 == 0):
            Matriz[i+1,i] = c
            Matriz[i,i+1] = c
        else:
            Matriz[i+1,i] = b
            Matriz[i,i+1] = b
            
    if PBC:
        Matriz[0,N-1] = b #PBC
        Matriz[N-1,0] = b #PBC
            
    return Matriz  

def Diagonalizar(Matriz):
    
    eig_vals, eig_vecs = np.linalg.eig(Matriz)
    eig_vals_sorted = np.sort(eig_vals)
    eig_vecs_sorted = eig_vecs[:, eig_vals.argsort()]
    
    N = Matriz.shape[0]
    D = np.identity(N)*eig_vals_sorted
    Q = np.asmatrix(eig_vecs_sorted)

    return D, Q, np.asmatrix(Matriz)

def Estado_Base(D,U):
    i = 0
    while D[i,i] <= 0:
        i = i +1
    indice = i #Ese es la energía del estado base:    
    Vec_Est_Base = U[:,indice]
    lambda_base = D[indice,indice]
    
    return Vec_Est_Base/np.linalg.norm(Vec_Est_Base), lambda_base #Retornemoslo normalizado

# Funciones Spread Complexity

In [9]:
def revisar_ortogonalidad(Lanczos_Matriz):
    N = Lanczos_Matriz.shape[0]
    
    for i in range(N):
        for j in range(N):
            Mensaje = "< u_" + str(i) + "| u_" + str(j) + "> = "
            valor = float(np.transpose( Lanczos_Matriz[:,i])@ Lanczos_Matriz[:,j])
            Mensaje = Mensaje + str(valor)
            print(Mensaje)

def revisar_tridiagonal(Lanczos_Matrix, a_coef, b_coef, H):
    N = Lanczos_Matrix.shape[0]
    
    for n in range(1,N - 1):
        Mensaje = "|| H|u_" + str(n) + "> - (a_" + str(n) + "|u_" + str(n) +"> + b_" +str(n+1) + "|u_" + str(n+1) + " + b_" +str(n) + "|u_" + str(n-1) + ">) || = "
        Mensaje = Mensaje + str(float(np.linalg.norm(H@Lanczos_Matrix[:,n] - ( a_coef[n]*Lanczos_Matrix[:,n] + b_coef[n+1]*Lanczos_Matrix[:,n+1] + b_coef[n]*Lanczos_Matrix[:,n-1]))))
        print(Mensaje)

def spread_complexity(tiempos, Lanczos_Basis, H):
    complejidad = np.array([])
    u_0 = Lanczos_Basis[:,0]

    for i in range(len(tiempos)):
        t = tiempos[i]
        u_t = np.dot(linalg.expm((-1j)*H*t), u_0)
        
        if(Lanczos_Basis.shape[0] == Lanczos_Basis.shape[1]): coefs_phi_t = np.linalg.inv(Lanczos_Basis)@u_t
        else: coefs_phi_t = np.linalg.pinv(Lanczos_Basis)@u_t
        
        c_t = 0
        for n in range(len(coefs_phi_t)):
            c_t = c_t + n*np.abs(coefs_phi_t[n])**2
        complejidad = np.append(complejidad, c_t)
        
    return complejidad

def Krylov_Basis(H_base_sitios, Vec_Est_Base):
    N = H_base_sitios.shape[0]
    
    Krylov_basis = np.asmatrix(np.zeros([N,N]))
    for n in range(N):
        Krylov_basis[:,n] = (H_base_sitios**n)@Vec_Est_Base
        
    return Krylov_basis